In [9]:
import os
import tiktoken
import openai  # Ben bu çalışmada, openai kütüphanesinin "openai==0.27.6" sürümünü kullandım.
from dotenv import load_dotenv
import warnings
from requests.packages.urllib3.exceptions import NotOpenSSLWarning

warnings.simplefilter('ignore', NotOpenSSLWarning)
load_dotenv()

# Daha önce azure üzerinde deploy ettiğim gpt-3.5 Turbo modelini kullanacağım.
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2023-05-15"

DEPLOYMENT_NAME =  os.environ.get("OPENAI_CHAT_DEPLOYMENT_NAME") 

In [2]:
def chat(prompt, model=DEPLOYMENT_NAME):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine= DEPLOYMENT_NAME,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

### Self-Consistency

Zincirleme düşünce (Chain of Thought - CoT) yöntemini geliştirerek, sorunlara birden fazla açıdan yaklaşım sağlamak ve en tutarlı yanıtı bulmak.

Nasıl Çalışır?
Çoklu Çıkarım Yolları: Bir soruna ilişkin farklı çıkarım yolları oluşturulur.
En Tutarlı Yanıtın Seçimi: Bu yolların her birinden elde edilen yanıtlar değerlendirilir ve en tutarlı olanı seçilir.
Örnek
Bir matematik problemi için, CoT yöntemiyle farklı çözüm yolları üretilir. Her bir yolun sonucu değerlendirilir ve en mantıklı ve doğru olan yanıt seçilir. Bu, özellikle karmaşık aritmetik problemlerinde etkili bir yöntemdir. 

[Reference](https://arxiv.org/abs/2203.11171)

![Self Consistency](./images/Self-Consistency.png)

In [3]:
prompt = f"""
Ben 12 yaşındayken kız kardeşim benim yaşımın yarısıydı. Şimdi 70 yaşındayım, kız kardeşim kaç yaşında?
"""

response = chat(prompt)
print(response)

Kız kardeşiniz şu anda 58 yaşında. 

Çözüm: 

Ben 12 yaşındayken kız kardeşiniz 12/2 = 6 yaşındaydı. 

Aradan geçen zaman: 70 - 12 = 58 yıl. 

Bu süre boyunca kız kardeşiniz de 58 - 6 = 52 yaşlandı. 

Dolayısıyla kız kardeşiniz şu anda 52 + 6 = 58 yaşında.


In [4]:
prompt = f"""
S: Bahçede 15 ağaç var. Bahçe işçileri bugün bahçeye ağaç dikecekler. İşleri bittikten sonra,
bahçede 21 ağaç olacak. Bahçe işçileri bugün kaç ağaç dikti?
C: Başlangıçta 15 ağaç var. Daha sonra 21 ağaç oluyor. Fark, dikilen ağaç sayısı olmalı.
Yani, 21 - 15 = 6 ağaç dikmişlerdir. Cevap 6.
S: Otoparkta 3 araba var ve 2 araba daha geliyor, otoparkta kaç araba olur?
C: Otoparkta zaten 3 araba var. 2 araba daha geliyor. Şimdi 3 + 2 = 5 araba var. Cevap 5.
S: Leah'in 32 çikolatası var ve kız kardeşinin 42 tane. Eğer 35 tanesini yerlerse, toplamda kaç çikolata kalmış olur?
C: Leah'in 32 çikolatası var ve Leah'in kız kardeşi 42 tane var. Bu, başlangıçta 32 + 42 = 74
çikolata olduğu anlamına gelir. 35 tanesi yenmiş. Yani toplamda hala 74 - 35 = 39 çikolata var. Cevap 39.
S: Jason'un 20 lolipopu vardı. Denny'ye bazı lolipopları verdi. Şimdi Jason'un 12 lolipopu var. Jason, Denny'ye kaç lolipop verdi?
C: Jason'un 20 lolipopu vardı. Şu anda sadece 12'si var, geri kalanını Denny'ye vermiş olmalı. Denny'ye verdiği
lolipop sayısı 20 - 12 = 8 lolipop olmalı. Cevap 8.
S: Shawn'un beş oyuncağı var. Noel'de annesinden ve babasından ikişer oyuncak aldı. Şimdi kaç oyuncağı var?
C: 5 oyuncağı var. Anneden 2 oyuncak aldı, o zaman 5 + 2 = 7 oyuncağı oldu. Sonra babadan 2 daha aldı, yani
toplamda 7 + 2 = 9 oyuncağı var. Cevap 9.
S: Sunucu odasında dokuz bilgisayar vardı. Pazartesiden perşembeye kadar her gün beşer bilgisayar yüklendi. Şimdi sunucu odasında kaç bilgisayar var?
C: Pazartesiden perşembeye 4 gün var. Her gün 5 bilgisayar eklendi. Bu, toplamda 4 * 5 = 20
bilgisayar eklendiği anlamına gelir. Başlangıçta 9 bilgisayar vardı, şimdi 9 + 20 = 29 bilgisayar var.
Cevap 29.
S: Michael'in 58 golf topu vardı. Salı günü 23 golf topu kaybetti. Çarşamba günü 2 tane daha kaybetti. Çarşamba gününün sonunda kaç golf topu kalmıştır?
C: Michael başlangıçta 58 topa sahipti. Salı günü 23 top kaybetti, o zaman onun 58 - 23 = 35 topu kalmıştır. Çarşamba
günü 2 daha kaybettiği için şimdi 35 - 2 = 33 topu var. Cevap 33.
S: Olivia'nın 23 doları var. Her biri 3 dolar olan beş bagel aldı. Ne kadar parası kaldı?
C: Her biri 3 dolardan 5 bagel aldı. Bu, 15 dolar harcadığı anlamına gelir. 8 doları kaldı.
S: Ben 12 yaşındayken kız kardeşim benim yaşımın yarısıydı. Şimdi 70 yaşındayım, kız kardeşim kaç yaşında?
C:
"""

response = chat(prompt)
print(response)

Benim yaşım 12 iken, kız kardeşim 6 yaşındaydı (yaşımın yarısı). Şimdi ben 70 yaşındayım, yani 70 - 12 = 58 yıl geçti. Kız kardeşim de 58 yıl daha yaşlandı, yani şimdi 6 + 58 = 64 yaşında olmalı. Cevap 64.


### Generated Knowledge Prompting

[Reference](https://arxiv.org/pdf/2110.08387.pdf)

![](./images/generated-knowledge-prompting.png)

In [5]:
prompt = f"""
Hangi ülke daha büyük, Kongo mu yoksa Güney Afrika mı?
"""

response = chat(prompt)
print(response)

Güney Afrika daha büyüktür. Toplam yüzölçümü 1.221.037 km² iken, Kongo'nun yüzölçümü 342.000 km²'dir.


In [6]:
prompt = f"""
Güney Afrika ve Kongo'nun büyüklükleri hakkında biraz bilgi verin:
"""

response = chat(prompt)
print(response)

Güney Afrika, Afrika kıtasının güneyinde yer alan bir ülkedir. Toplam yüzölçümü 1.221.037 kilometrekaredir. Bu, Türkiye'nin yaklaşık 3 katı büyüklüğündedir. Güney Afrika, 9 eyaletten oluşur ve Johannesburg, Cape Town ve Pretoria gibi büyük şehirleri vardır.

Kongo, Afrika kıtasının orta bölgesinde yer alan bir ülkedir. Toplam yüzölçümü 2.345.410 kilometrekaredir. Bu, Türkiye'nin yaklaşık 6 katı büyüklüğündedir. Kongo, iki ayrı ülkeden oluşur: Kongo Cumhuriyeti ve Demokratik Kongo Cumhuriyeti. Başkentleri sırasıyla Brazzaville ve Kinshasa'dır. Kongo, dünyanın en büyük yağmur ormanlarından biri olan Kongo Ormanı'na ev sahipliği yapar.


In [7]:
prompt = f"""
Güney Afrika, Afrika kıtasının en güney kısmında yer alan bir ülkedir. Yaklaşık 1.219.090 kilometrekarelik bir toplam karasal alana sahiptir ve bu da onu dünyanın 25. en büyük ülkesi yapar. Güney Afrika, Namibya, Botsvana, Zimbabve, Mozambik ve Svaziland ile sınırlıdır ve Atlantik ve Hint Okyanusları boyunca 2.798 kilometre uzunluğunda bir sahil şeridine sahiptir.

Öte yandan, Demokratik Kongo Cumhuriyeti, aynı zamanda Kongo-Kinşasa olarak da bilinir, Orta Afrika'da yer alan bir ülkedir. Yaklaşık 2.344.858 kilometrekarelik bir toplam karasal alana sahiptir ve bu da onu dünyanın 11. en büyük ülkesi yapar. Kongo, Angola, Zambiya, Tanzanya ve Uganda da dahil olmak üzere dokuz ülke ile sınırlıdır ve Atlantik Okyanusu boyunca 37 kilometre uzunluğunda bir sahil şeridine sahiptir.

Büyüklük açısından, Kongo, Güney Afrika'dan neredeyse iki kat daha büyüktür. Ancak, Güney Afrika'nın daha yüksek bir nüfus yoğunluğu vardır; tahmini 59,3 milyon kişilik bir nüfusa sahipken, Kongo'nun tahmini nüfusu 89,6 milyon kişidir.

Hangi ülke daha büyük, Kongo mu yoksa Güney Afrika mı?
"""

response = chat(prompt)
print(response)

Kongo, Güney Afrika'dan daha büyüktür.


### Tree of Thoughts (ToT)

![](./images/tot.png)


- [References 1](https://arxiv.org/abs/2305.10601)
- [References 2](https://arxiv.org/abs/2305.08291)
- [Prompt reference](https://github.com/dave1010/tree-of-thought-prompting)


In [8]:
prompt = f"""
Beş farklı uzmanın bu soruyu cevapladığını hayal edin.
Tüm uzmanlar düşünme süreçlerinin 1 adımını yazacak,
sonra bunu grupla paylaşacaklar.
Daha sonra tüm uzmanlar bir sonraki adıma geçecek, ve böyle devam edecek.
Eğer herhangi bir uzman, herhangi bir noktada yanıldığını fark ederse, o zaman ayrılacak.
Soru şu...

Hangi ülke daha büyük, Kongo mu yoksa Güney Afrika mı?

Bana son bir sonuç verin.
"""

response = chat(prompt)
print(response)

Sonuç olarak, uzmanlar arasında farklı düşünceler ve tartışmalar olsa da, çoğunlukla Güney Afrika'nın Kongo'dan daha büyük olduğu sonucuna varılmıştır. Ancak, bu sonuç kesin değildir ve uzmanlar arasında farklı görüşler olabilir.
